<a href="https://colab.research.google.com/github/18Srikar/dm_hw3/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [6]:
import time
import os
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances_argmin_min
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Load the feature and target datasets
features_df = pd.read_csv('data.csv', header=None)
labels_df = pd.read_csv('label.csv', header=None)
labels_df.columns = ['label']

# Combine features and target labels
full_dataset = pd.concat([features_df, labels_df], axis=1)
processed_data = full_dataset

# Extract features and labels
input_features = processed_data.iloc[:, :-1].values
true_labels = processed_data.iloc[:, -1].values

# Determine the number of unique clusters
total_clusters = full_dataset['label'].nunique()

In [8]:

# Function to initialize centroids for K-means
def init_centroids(data, k_clusters):
    return data[np.random.choice(len(data), k_clusters, replace=False)]

# Function to calculate SSE (Sum of Squared Errors)
def calc_sse(data, cluster_assignments, centroids):
    return np.sum(np.linalg.norm(data - centroids[cluster_assignments], axis=1)**2)

# Function to update centroids
def recalculate_centroids(data, cluster_assignments, k_clusters):
    return np.array([data[cluster_assignments == cluster].mean(axis=0) for cluster in range(k_clusters)])

# Function to assign clusters based on distances
def assign_to_clusters(data, centroids):
    distances = np.linalg.norm(data[:, np.newaxis] - centroids, axis=2)
    return np.argmin(distances, axis=1)

# Standard K-means clustering implementation
def kmeans(data, k_clusters, max_iterations=100):
    centroids = init_centroids(data, k_clusters)

    for _ in range(max_iterations):
        previous_centroids = centroids.copy()
        cluster_labels = assign_to_clusters(data, centroids)
        centroids = recalculate_centroids(data, cluster_labels, k_clusters)

        # Check for convergence
        if np.all(previous_centroids == centroids):
            break

    final_sse = calc_sse(data, cluster_labels, centroids)
    return centroids, cluster_labels, final_sse

In [9]:


# Distance metric implementations
def euclidean_dist(point_set1, point_set2):
    return np.linalg.norm(point_set1[:, np.newaxis] - point_set2, axis=2)

def cosine_dist(point_set1, point_set2):
    dot_product = np.dot(point_set1, point_set2.T)
    norm1 = np.linalg.norm(point_set1, axis=1)[:, np.newaxis]
    norm2 = np.linalg.norm(point_set2, axis=1)
    return 1 - dot_product / (norm1 * norm2)

def jaccard_dist(point_set1, point_set2):
    intersection = np.minimum(point_set1[:, np.newaxis], point_set2).sum(axis=2)
    union = np.maximum(point_set1[:, np.newaxis], point_set2).sum(axis=2)
    return 1 - intersection / union

# Custom K-means implementation with different distance metrics
def custom_kmeans(data, k_clusters, dist_func, max_iters=100):
    num_samples = data.shape[0]
    centroids = data[np.random.choice(num_samples, k_clusters, replace=False)]

    for _ in range(max_iters):
        # Compute distance matrix
        dist_matrix = dist_func(data, centroids)
        cluster_assignments = np.argmin(dist_matrix, axis=1)

        # Update centroids
        updated_centroids = np.array([data[cluster_assignments == cluster].mean(axis=0) for cluster in range(k_clusters)])

        if np.all(centroids == updated_centroids):
            break
        centroids = updated_centroids

    final_sse = np.sum(np.min(dist_func(data, centroids)**2, axis=1))
    return cluster_assignments, centroids, final_sse

In [10]:
# Evaluate different distance functions for clustering
distance_functions = [euclidean_dist, cosine_dist, jaccard_dist]
method_names = ['Euclidean K-means', 'Cosine K-means', 'Jaccard K-means']
results = {}

for distance_func, method in zip(distance_functions, method_names):
    cluster_labels, centroids, sse = custom_kmeans(input_features, total_clusters, distance_func)
    results[method] = {'SSE': sse}

for method, result in results.items():
    print(f"{method} - SSE: {result['SSE']:.2f}")


Euclidean K-means - SSE: 25321529890.34
Cosine K-means - SSE: 682.25
Jaccard K-means - SSE: 3723.96


In [11]:
def dist_euclidean(x1, x2):
    return np.linalg.norm(x1 - x2)

def sim_cosine(x1, x2):
    norm_x1 = np.linalg.norm(x1)
    norm_x2 = np.linalg.norm(x2)
    if norm_x1 == 0 or norm_x2 == 0:
        return 1
    return 1 - np.dot(x1, x2) / (norm_x1 * norm_x2)

def sim_jaccard(x1, x2):
    intersection = np.sum(np.minimum(x1, x2))
    union = np.sum(np.maximum(x1, x2))
    if union == 0:
        return 1
    return 1 - intersection / union
def majority_vote_kmeans(data_points, num_clusters, distance_metric, max_iters=100):
    num_samples, num_features = data_points.shape
    cluster_centers = data_points[np.random.choice(num_samples, num_clusters, replace=False)]
    cluster_assignments = np.zeros(num_samples)

    for _ in range(max_iters):
        # Assign each data point to the nearest cluster center
        for i in range(num_samples):
            distances = [distance_metric(data_points[i], center) for center in cluster_centers]
            cluster_assignments[i] = np.argmin(distances)

        # Update cluster centers based on the current assignments
        for cluster_id in range(num_clusters):
            points_in_cluster = cluster_assignments == cluster_id
            if np.any(points_in_cluster):
                cluster_centers[cluster_id] = np.mean(data_points[points_in_cluster], axis=0)

    # Perform majority vote for assigning labels to clusters
    cluster_labels = np.zeros(num_clusters)
    for cluster_id in range(num_clusters):
        points_in_cluster = cluster_assignments == cluster_id
        if np.any(points_in_cluster):
            dominant_label = np.argmax(np.bincount(true_labels[points_in_cluster].astype(int)))
            cluster_labels[cluster_id] = dominant_label

    # Assign final predictions based on majority vote
    final_predictions = np.array([cluster_labels[int(cluster)] for cluster in cluster_assignments])
    accuracy = np.sum(final_predictions == true_labels) / len(true_labels)

    return accuracy


# List of distance metrics
distance_metrics = [dist_euclidean, sim_cosine, sim_jaccard]
method_names = ['Euclidean', 'Cosine', 'Jaccard']

# Store accuracy results
accuracy_results = {}

for metric, method_name in zip(distance_metrics, method_names):
    accuracy = majority_vote_kmeans(input_features, total_clusters, metric)
    accuracy_results[method_name] = accuracy

# Print results
for method_name, accuracy in accuracy_results.items():
    print(f"{method_name} K-means with Majority Vote - Accuracy: {accuracy * 100:.2f}%")


Euclidean K-means with Majority Vote - Accuracy: 63.52%
Cosine K-means with Majority Vote - Accuracy: 59.19%
Jaccard K-means with Majority Vote - Accuracy: 60.67%


In [12]:
import time
import numpy as np

def kmeans_with_convergence_criteria(input_data, num_clusters, distance_func, max_iterations=500, convergence_threshold=1e-4):
    num_samples, num_features = input_data.shape
    cluster_centers = input_data[np.random.choice(num_samples, num_clusters, replace=False)]

    for iteration in range(max_iterations):
        # Compute distances from data points to each cluster center
        distances = np.array([distance_func(input_data, center) for center in cluster_centers])

        # Assign each point to the nearest cluster
        cluster_assignments = np.argmin(distances, axis=0)

        # Compute new centroids as the mean of points in each cluster
        updated_centers = np.array([
            input_data[cluster_assignments == cluster_id].mean(axis=0)
            for cluster_id in range(num_clusters)
        ])

        # Check for convergence
        if np.all(np.abs(updated_centers - cluster_centers) < convergence_threshold):
            break

        cluster_centers = updated_centers

    # Calculate final SSE
    final_sse = np.sum([
        distance_func(input_data[i], cluster_centers[cluster_assignments[i]])**2
        for i in range(num_samples)
    ])

    return cluster_assignments, cluster_centers, iteration + 1, final_sse

# Distance metrics
def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2, axis=-1)

def cosine_distance(point1, point2):
    dot_product = np.dot(point1, point2)
    norm_point1 = np.linalg.norm(point1, axis=-1)
    norm_point2 = np.linalg.norm(point2)
    return 1 - dot_product / (norm_point1 * norm_point2)

def jaccard_distance(point1, point2):
    intersection = np.minimum(point1, point2).sum(axis=-1)
    union = np.maximum(point1, point2).sum(axis=-1)
    return 1 - intersection / union

# Metrics and results
distance_functions = [euclidean_distance, cosine_distance, jaccard_distance]
method_names = ['Euclidean', 'Cosine', 'Jaccard']
clustering_results = {}

# Perform clustering for each distance metric
for metric_func, method_name in zip(distance_functions, method_names):
    start_time = time.time()

    assignments, centers, iterations, sse = kmeans_with_convergence_criteria(input_features, total_clusters, metric_func)

    end_time = time.time()

    clustering_results[method_name] = {
        'Cluster Assignments': assignments,
        'Cluster Centers': centers,
        'Iterations Taken': iterations,
        'Final SSE': sse,
        'Execution Time': end_time - start_time
    }

# Print results
for method, result in clustering_results.items():
    print(f"{method: <15} | Iterations: {result['Iterations Taken']: <5} | SSE: {result['Final SSE']:.2f} | Time: {result['Execution Time']:.4f} seconds")


Euclidean       | Iterations: 39    | SSE: 25409131535.93 | Time: 24.7406 seconds
Cosine          | Iterations: 59    | SSE: 699.60 | Time: 54.8771 seconds
Jaccard         | Iterations: 41    | SSE: 3674.82 | Time: 28.9271 seconds


In [13]:
import numpy as np

def kmeans_with_termination_criteria(data, num_clusters, distance_function, max_iterations=100, convergence_threshold=1e-4, termination_condition='no_change'):
    num_samples, num_features = data.shape
    centroids = data[np.random.choice(num_samples, num_clusters, replace=False)]
    prev_sse = float('inf')
    stop_reason = None

    for iteration in range(max_iterations):
        # Calculate distances to each centroid
        distances = np.array([distance_function(data, centroid) for centroid in centroids])
        if distances.ndim == 1:
            distances = distances.reshape(1, -1)
        elif distances.shape[0] != num_samples:
            distances = distances.T

        # Assign each point to the nearest cluster
        cluster_assignments = np.argmin(distances, axis=1)

        # Update centroids
        new_centroids = []
        for cluster_id in range(num_clusters):
            cluster_points = data[cluster_assignments == cluster_id]
            if len(cluster_points) > 0:
                new_centroids.append(cluster_points.mean(axis=0))
            else:
                new_centroids.append(centroids[cluster_id])  # Keep the old centroid if the cluster is empty
        updated_centroids = np.array(new_centroids)

        # Compute SSE (Sum of Squared Errors)
        sse = np.sum([
            distance_function(data[i:i + 1], updated_centroids[cluster_assignments[i]])**2
            for i in range(num_samples)
        ])

        # Check termination conditions
        if termination_condition == 'no_change' and np.all(np.abs(updated_centroids - centroids) < convergence_threshold):
            stop_reason = 'converged_with_no_change'
            break
        elif termination_condition == 'sse_increase' and sse > prev_sse:
            stop_reason = 'sse_increased'
            break
        elif termination_condition == 'max_iterations' and iteration == max_iterations - 1:
            stop_reason = 'max_iterations_reached'
            break

        # Update for next iteration
        centroids = updated_centroids
        prev_sse = sse

    if stop_reason is None:
        stop_reason = 'max_iterations_reached'

    return cluster_assignments, centroids, iteration + 1, sse, stop_reason

# Distance metrics
def euclidean_distance(X, Y):
    return np.linalg.norm(X - Y, axis=1)

def cosine_distance(X, Y):
    dot_product = np.sum(X * Y, axis=1)
    norm_X = np.linalg.norm(X, axis=1)
    norm_Y = np.linalg.norm(Y)
    return 1 - dot_product / (norm_X * norm_Y)

def jaccard_distance(X, Y):
    intersection = np.sum(np.minimum(X, Y), axis=1)
    union = np.sum(np.maximum(X, Y), axis=1)
    return 1 - intersection / union

# Define distance metrics and their names
distance_functions = [euclidean_distance, cosine_distance, jaccard_distance]
distance_function_names = ['Euclidean', 'Cosine', 'Jaccard']

# Store SSE results
sse_results = {}

# Iterate through each distance metric and apply different termination conditions
for distance_function, function_name in zip(distance_functions, distance_function_names):
    # Termination based on no centroid change
    cluster_labels, centroids, iterations, sse, stop_reason = kmeans_with_termination_criteria(
        input_features, total_clusters, distance_function, termination_condition='no_change'
    )
    sse_results[f"{function_name}_No_Change"] = sse

    # Termination based on SSE increase
    cluster_labels, centroids, iterations, sse, stop_reason = kmeans_with_termination_criteria(
        input_features, total_clusters, distance_function, termination_condition='sse_increase'
    )
    sse_results[f"{function_name}_SSE_Increase"] = sse

    # Termination based on maximum iterations
    cluster_labels, centroids, iterations, sse, stop_reason = kmeans_with_termination_criteria(
        input_features, total_clusters, distance_function, termination_condition='max_iterations'
    )
    sse_results[f"{function_name}_Max_Iterations"] = sse

# Display SSE results for each method and condition
for method_name, sse in sse_results.items():
    formatted_method = method_name.replace("_", " ").title()
    print(f"SSE of {formatted_method}: {sse:.2f}\n")




SSE of Euclidean No Change: 25523380485.88

SSE of Euclidean Sse Increase: 25408123696.02

SSE of Euclidean Max Iterations: 25502248446.17

SSE of Cosine No Change: 687.78

SSE of Cosine Sse Increase: 696.70

SSE of Cosine Max Iterations: 685.07

SSE of Jaccard No Change: 3660.84

SSE of Jaccard Sse Increase: 3659.99

SSE of Jaccard Max Iterations: 3731.01

